In [522]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

os.getcwd()

'/home/cseadmin/undergrad1/dz/GISCUP2021/data_process'

In [523]:
df_link=pd.read_csv("../data/train/train_link/20200801.csv", nrows=1000)
df_cross=pd.read_csv("../data/train/train_cross/20200801.csv", nrows=1000)

In [524]:
df_link

,order_id,link_id,link_time,link_ratio,link_current_status,link_arrival_status
0,5966280,157298,4.5000,1.0000,1,0
1,5966280,511622,2.8552,1.0000,1,1
2,5966280,524542,2.2345,1.0000,1,0
3,5966280,97293,3.6000,1.0000,1,1
4,5966280,273845,3.2276,1.0000,1,1
...,...,...,...,...,...,...
995,4144356,254845,2.5297,1.0000,1,1
996,4144356,397390,6.4664,0.7254,1,0
997,4516865,501423,5.6415,0.3660,0,0
998,4516865,74783,9.0000,1.0000,0,0


In [525]:
df_cross=df_cross.dropna()

df_cross

,order_id,cross id,cross_time
0,5966280,47128_204460,23.0
1,5966280,624101_153565,30.0
2,5966280,30464_397883,26.0
3,5966280,160457_527119,18.0
4,5966280,597673_507113,30.0
...,...,...,...
994,4812329,224111_178502,15.0
995,5116124,544005_316701,9.0
996,5116124,146672_155592,30.0
997,5116124,1813_368869,30.0


## 1. 添加 cross_flag

In [526]:
df_link["cross_flag"]=0

df_link

,order_id,link_id,link_time,link_ratio,link_current_status,link_arrival_status,cross_flag
0,5966280,157298,4.5000,1.0000,1,0,0
1,5966280,511622,2.8552,1.0000,1,1,0
2,5966280,524542,2.2345,1.0000,1,0,0
3,5966280,97293,3.6000,1.0000,1,1,0
4,5966280,273845,3.2276,1.0000,1,1,0
...,...,...,...,...,...,...,...
995,4144356,254845,2.5297,1.0000,1,1,0
996,4144356,397390,6.4664,0.7254,1,0,0
997,4516865,501423,5.6415,0.3660,0,0,0
998,4516865,74783,9.0000,1.0000,0,0,0


## 2. 获取每行 index+2 的 link_id (这里 O(n) 的操作不知道为什么非常耗时)

In [527]:
def get_next_2_link_id(index):
    if index+2<=df_link.last_valid_index():
        return df_link.loc[index+2, "link_id"]
    return -1

In [528]:
df_link["next_2_link_id"]=df_link.apply(lambda row: get_next_2_link_id(row.name), axis=1)

df_link

,order_id,link_id,link_time,link_ratio,link_current_status,link_arrival_status,cross_flag,next_2_link_id
0,5966280,157298,4.5000,1.0000,1,0,0,524542
1,5966280,511622,2.8552,1.0000,1,1,0,97293
2,5966280,524542,2.2345,1.0000,1,0,0,273845
3,5966280,97293,3.6000,1.0000,1,1,0,556273
4,5966280,273845,3.2276,1.0000,1,1,0,73842
...,...,...,...,...,...,...,...,...
995,4144356,254845,2.5297,1.0000,1,1,0,501423
996,4144356,397390,6.4664,0.7254,1,0,0,74783
997,4516865,501423,5.6415,0.3660,0,0,0,57402
998,4516865,74783,9.0000,1.0000,0,0,0,-1


## 3. 插值

In [529]:
def insert_cross(row):
    global df_link

    link1, link2=list(map(int, row["cross id"].split("_")))
    index=df_link.loc[(df_link["order_id"]==row["order_id"]) & (df_link["link_id"]==link1) & (df_link["next_2_link_id"]==link2)]

    if index.empty:
        return
    
    index=index.iloc[0].name
    # print(index)

    df1=df_link.loc[:index]
    df2=df_link.loc[index+1:]

    max_index=max(df_link.index)
    df1.loc[max_index+1]=[row["order_id"], row["cross id"], row["cross_time"], -1, -1, -1, 1, -1]

    df_link=df1.append(df2)

In [530]:
import time

start=time.time()
df_cross.apply(insert_cross, axis=1)
end=time.time()

print(f"Time Cost = {end-start}s")

df_link.head(15)

Time Cost = 0.9336776733398438s


,order_id,link_id,link_time,link_ratio,link_current_status,link_arrival_status,cross_flag,next_2_link_id
0,5966280,157298,4.5000,1.0,1,0,0,524542
1,5966280,511622,2.8552,1.0,1,1,0,97293
2,5966280,524542,2.2345,1.0,1,0,0,273845
3,5966280,97293,3.6000,1.0,1,1,0,556273
4,5966280,273845,3.2276,1.0,1,1,0,73842
5,5966280,556273,3.7241,1.0,1,1,0,101675
6,5966280,73842,2.4828,1.0,1,1,0,325444
7,5966280,101675,2.4828,1.0,1,1,0,134737
8,5966280,325444,0.7043,1.0,1,1,0,611534
9,5966280,134737,3.6000,1.0,1,1,0,47128


In [531]:
df_link=df_link.drop(columns=["next_2_link_id"])

df_link

,order_id,link_id,link_time,link_ratio,link_current_status,link_arrival_status,cross_flag
0,5966280,157298,4.5000,1.0000,1,0,0
1,5966280,511622,2.8552,1.0000,1,1,0
2,5966280,524542,2.2345,1.0000,1,0,0
3,5966280,97293,3.6000,1.0000,1,1,0
4,5966280,273845,3.2276,1.0000,1,1,0
...,...,...,...,...,...,...,...
995,4144356,254845,2.5297,1.0000,1,1,0
996,4144356,397390,6.4664,0.7254,1,0,0
997,4516865,501423,5.6415,0.3660,0,0,0
998,4516865,74783,9.0000,1.0000,0,0,0
